In [2]:
import pandas as pd
from rapidfuzz import fuzz
import re

In [3]:
# Changed to index_col=[0] to prevent Unnamed: X columns
df = pd.read_csv("company_website_second_round_with_additional_firms.csv", low_memory=True, index_col=[0])
df = df.drop(columns=[col for col in df.columns if col.startswith('Unnamed:')]) # If th

non_temporal_df = df.iloc[:, 0:13] # Will need to change indexing if more non-temporal columns are added
df = df.drop(df.columns[0:13], axis=1)
assert not any(re.match(r"\d{4}-\d{2}", col) for col in non_temporal_df.columns), "Columns with the pattern 'YYYY-MM' are present in the non-temporal dataframe"
assert all(re.match(r"\d{4}-\d{2}", col) for col in df.columns), "Not all columns follow the 'YYYY-MM' pattern"

/tmp/ipykernel_197/3174632621.py:2: DtypeWarning: Columns (300,304,325,328) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("company_website_second_round_with_additional_firms.csv", low_memory=True, index_col=[0])


In [4]:
df.shape

(5189, 325)

In [5]:
def compute_hash(string):
    ans = 0
    for i in range(0, min(len(string), 8)):
        ans += ord(string[i]) * (7**i)
    return ans

In [6]:
table = {}
saved = 0
considering = 0
for index, row in enumerate(df.itertuples()):
    if index % 100 == 0:
        print(f"Finished processing {index} out of {len(df.index)} rows.")
    # print(len(row))
    for column_index, value in enumerate(row):
        if pd.isna(value) or type(value) == int or type(value) == float:
            continue
        value_hash = compute_hash(value)
        
        if not(value_hash in table):
            table[value_hash] = []

        max_similarity = 0
        for existing_value in table[value_hash]:
            max_similarity = max(max_similarity, fuzz.ratio(existing_value, value))

        if max_similarity < 0.95:
            table[value_hash].append(value)
            considering += 1
        else:
            saved += 1
            # print(index, column_index)
            df.iloc[index, column_index - 1] = -1 # subtracting 1 from the column_index because the first one is probably something irrelevant
            
print(saved, considering)

Finished processing 0 out of 5189 rows.
Finished processing 100 out of 5189 rows.
Finished processing 200 out of 5189 rows.
Finished processing 300 out of 5189 rows.
Finished processing 400 out of 5189 rows.
Finished processing 500 out of 5189 rows.
Finished processing 600 out of 5189 rows.
Finished processing 700 out of 5189 rows.
Finished processing 800 out of 5189 rows.
Finished processing 900 out of 5189 rows.
Finished processing 1000 out of 5189 rows.
Finished processing 1100 out of 5189 rows.
Finished processing 1200 out of 5189 rows.
Finished processing 1300 out of 5189 rows.
Finished processing 1400 out of 5189 rows.
Finished processing 1500 out of 5189 rows.
Finished processing 1600 out of 5189 rows.
Finished processing 1700 out of 5189 rows.
Finished processing 1800 out of 5189 rows.
Finished processing 1900 out of 5189 rows.
Finished processing 2000 out of 5189 rows.
Finished processing 2100 out of 5189 rows.
Finished processing 2200 out of 5189 rows.
Finished processing 230

In [7]:
actual_text_count = 0

for index, row in enumerate(df.itertuples()):
    if index % 100 == 0:
        print(f"Finished processing {index} out of {len(df.index)} rows.")
    # print(len(row))
    for column_index, value in enumerate(row):
        if pd.isna(value) or type(value) == int or type(value) == float:
            continue
        actual_text_count += 1

print(actual_text_count)

Finished processing 0 out of 5189 rows.
Finished processing 100 out of 5189 rows.
Finished processing 200 out of 5189 rows.
Finished processing 300 out of 5189 rows.
Finished processing 400 out of 5189 rows.
Finished processing 500 out of 5189 rows.
Finished processing 600 out of 5189 rows.
Finished processing 700 out of 5189 rows.
Finished processing 800 out of 5189 rows.
Finished processing 900 out of 5189 rows.
Finished processing 1000 out of 5189 rows.
Finished processing 1100 out of 5189 rows.
Finished processing 1200 out of 5189 rows.
Finished processing 1300 out of 5189 rows.
Finished processing 1400 out of 5189 rows.
Finished processing 1500 out of 5189 rows.
Finished processing 1600 out of 5189 rows.
Finished processing 1700 out of 5189 rows.
Finished processing 1800 out of 5189 rows.
Finished processing 1900 out of 5189 rows.
Finished processing 2000 out of 5189 rows.
Finished processing 2100 out of 5189 rows.
Finished processing 2200 out of 5189 rows.
Finished processing 230

In [15]:
def get_phrases(corpus, length=20, split=15):
    total_words = [word.lower() for word in corpus.split()]
    phrases = []
    
    for i in range(0, len(total_words), split):
        right_idx = min(len(total_words), i + length)
        phrases.append(" ".join(total_words[i:right_idx]))
    
    return phrases

In [16]:
from sentence_transformers import SentenceTransformer, util
embeddings_model = SentenceTransformer("all-MiniLM-L6-v2")

In [18]:
keyword_files = [
    {"keyword": "antiforeign", "file": "antiforeign_revised_2_phrases.json"},
    {"keyword": "military", "file": "military_revised_phrases.json"},
    {"keyword": "national_pride", "file": "national_pride_revised_phrases.json"},
    {"keyword": "quality", "file": "quality_revised_phrases.json"},
    {"keyword": "revival", "file": "revival_phrases.json"},
    {"keyword": "jobs", "file": "jobs_revised_phrases.json"},
    {"keyword": "labor", "file": "labor_revised_phrases.json"}
]

In [ ]:
import json
import torch
import numpy as np
import time

for keyword_file in keyword_files:
    keyword = keyword_file["keyword"]
    filename = keyword_file["file"]

    existing_phrases_file = open("antiforeign_revised_2_phrases.json", "r")
    existing_phrases = json.loads(existing_phrases_file.read())[:10]
    existing_embedded = embeddings_model.encode(existing_phrases, convert_to_tensor=True)

    counter = 0
    time_last = time.time()
    similar_phrases = []
    
    non_llm_map = {}
    keyword = "antiforeign"
    total_phrase_count = 0
    
    def count_phrases(text):
        global time_last
        global counter 
        global similar_phrases
        global base_embedded
        global total_phrase_count
        
        if pd.isna(text) or type(text) == float or type(text) == int:
            return text
            
        phrases = get_phrases(text)
        if len(phrases) == 0:
            phrases = [text]
    
        phrase_count = 0
        phrases_embedded = embeddings_model.encode(phrases, convert_to_tensor=True)
        
        # Compute cosine-similarities for each sentence with each other sentence
        cosine_scores = util.cos_sim(existing_embedded, phrases_embedded)
        
        # Find the pairs with the highest cosine similarity scores
        for j in range(cosine_scores.shape[1]): # Iterating through all of the corpus phrases
            for i in range(cosine_scores.shape[0]): # Iterating through all of the existing original phrases
                if cosine_scores[i][j] > 0.4:
                    phrase_count += 1
                    total_phrase_count += 1
                    break # Already found an existing original phrase that is similar enough - break
        
        counter += 1
    
        if counter % 500 == 0:
            print(f"Processed {counter} cells in {time.time() - time_last}")
            time_last = time.time()
        
        return phrase_count

    all_similar_phrases = {}
    print("Current keyword being processed: ", keyword)
    df2 = df.map(count_phrases)
    all_similar_phrases[keyword] = similar_phrases
    df2.to_csv(f"count_csvs_4/{keyword}_simsearch_nofix_2.csv")
    df2 = df2.drop(columns=[col for col in df2.columns if col.startswith('Unnamed:')]) # If th
    for index, row in df2.iterrows():
        prev_value = None
        for col in reversed(df2.columns):
            value = row[col]
            if pd.isna(value) or value == -1 or value == 0:
                if prev_value is not None:
                    df2.at[index, col] = prev_value
            elif value == -1:
                df2.at[index, col] = 0
            else:
                prev_value = value
    print(df2.sum())
    df2.to_csv(f"count_csvs_4/prelim_{keyword}_simsearch4.csv")
    

Current keyword being processed:  antiforeign
Processed 500 cells in 13.90708589553833
Processed 1000 cells in 15.090537071228027
Processed 1500 cells in 13.737221956253052
Processed 2000 cells in 13.054450750350952
Processed 2500 cells in 14.681780338287354
Processed 3000 cells in 12.620123386383057
Processed 3500 cells in 12.826378583908081
Processed 4000 cells in 13.61329460144043
Processed 4500 cells in 15.889739274978638
Processed 5000 cells in 12.73293399810791
Processed 5500 cells in 11.786227941513062
Processed 6000 cells in 11.5247323513031
Processed 6500 cells in 9.94400954246521
Processed 7000 cells in 9.454385995864868
Processed 7500 cells in 8.101589679718018
Processed 8000 cells in 9.298517227172852
Processed 8500 cells in 8.685803890228271
Processed 9000 cells in 8.012553453445435
Processed 9500 cells in 6.9875335693359375
Processed 10000 cells in 7.3401923179626465
Processed 10500 cells in 7.500226736068726
Processed 11000 cells in 6.128046035766602
Processed 11500 cell